In [48]:
import os
import pandas as pd
import math
import numpy as np
import tensorflow as tf

from datetime import datetime
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras import backend as K
from keras import regularizers
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.metrics import roc_auc_score

K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

# The training set just has blanks instead of 0s
train_labels = pd.read_csv("CheXpert-v1.0-small/train.csv").fillna(0)
validation_labels = pd.read_csv('CheXpert-v1.0-small/valid.csv')

In [3]:
# Filter out Lateral images.  We'll train two models -> one for lateral and one for frontal

frontal_train_labels = train_labels[train_labels['Frontal/Lateral'] == 'Frontal']
frontal_validation_labels = validation_labels[validation_labels['Frontal/Lateral'] == 'Frontal']

# Filter out uncertains in the training dataset.  There are no uncertains in the validation dataset.
frontal_train_labels = frontal_train_labels[frontal_train_labels["Lung Opacity"] != -1.0]

In [ ]:
frontal_train_labels.head()

In [ ]:
frontal_validation_labels.head()

In [13]:
frontal_train_labels.describe()

,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
count,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000,186596.000000
mean,60.629365,0.090967,-0.005177,0.087510,0.504893,0.031474,0.200149,-0.060596,-0.059390,0.006972,0.080173,0.357596,0.003848,0.036276,0.558115
std,17.821530,0.287562,0.317897,0.386847,0.499977,0.203722,0.532046,0.433404,0.322723,0.553443,0.317387,0.569870,0.148507,0.200450,0.506179
min,0.000000,0.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,62.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,74.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,90.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
frontal_validation_labels.describe()

,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
count,202.000000,202.000000,202.000000,202.000000,202.000000,202.00000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.00000,202.0,202.000000
mean,60.816832,0.128713,0.519802,0.326733,0.579208,0.00495,0.207921,0.158416,0.039604,0.371287,0.034653,0.316832,0.00495,0.0,0.490099
std,18.336303,0.335714,0.500849,0.470184,0.494913,0.07036,0.406828,0.366038,0.195511,0.484349,0.183355,0.466397,0.07036,0.0,0.501144
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
25%,48.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
50%,62.500000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000
75%,74.750000,0.000000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.0,1.000000
max,90.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.0,1.000000


In [4]:
train_datagen = datagen.flow_from_dataframe(dataframe=frontal_train_labels,
                                            directory=".",
                                            x_col="Path",
                                            y_col=['Lung Opacity'],
                                            class_mode = "raw",
                                            color_mode='grayscale',
                                            target_size=(100, 100),
                                            batch_size=32)
validation_datagen = datagen.flow_from_dataframe(dataframe=frontal_validation_labels,
                                            directory=".",
                                            x_col="Path",
                                            y_col=['Lung Opacity'],
                                            class_mode = "raw",
                                            color_mode='grayscale',
                                            target_size=(100, 100),
                                            batch_size=32)

Found 186596 validated image filenames.
Found 202 validated image filenames.


In [40]:
# metrics functions

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
classifier = Sequential()

# L2 regularization uses the sum of the squares of the weights
l2_regularization_constant = 0

# 200x200 input
# Input is kinda large, so we select larger filters for the first layer to decrease 
# size of feature maps (and hopefully speed up training).

# Input: 100 x 100 x 1
classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), use_bias=False))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 96 x 96 x 32
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 94 x 94 x 64
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 47 x 47 x 64
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 45 x 45 x 128
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 43 x 43 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 21 x 21 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 19 x 19 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 17 x 17 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 8 x 8 x 128
classifier.add(Flatten())

# Input: 8192 x 1024
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer=Adam(learning_rate=0.002), loss='binary_crossentropy', metrics=[precision_m, recall_m, f1_m])

mc = ModelCheckpoint('trial_5/weights{epoch:04d}.h5', 
                                     save_weights_only=True, period=5)

os.makedirs('trial_5')

# too many epochs mean overfitting, not enough epochs mean underfitting
classifier.fit_generator(
    train_datagen,
    steps_per_epoch=5280,
    epochs=120,
    validation_data=validation_datagen,
    validation_steps=800,
    workers=4,
    verbose=2,
    callbacks=[mc])

Epoch 1/120
 - 1864s - loss: 0.7115 - precision_m: 0.5686 - recall_m: 0.7451 - f1_m: 0.6313 - val_loss: 0.7452 - val_precision_m: 0.7723 - val_recall_m: 0.1620 - val_f1_m: 0.2597
Epoch 2/120
 - 1870s - loss: 0.6623 - precision_m: 0.5871 - recall_m: 0.7368 - f1_m: 0.6452 - val_loss: 0.6471 - val_precision_m: 0.6701 - val_recall_m: 0.8780 - val_f1_m: 0.7550
Epoch 3/120
 - 1858s - loss: 0.6592 - precision_m: 0.5968 - recall_m: 0.7149 - f1_m: 0.6422 - val_loss: 0.6462 - val_precision_m: 0.7793 - val_recall_m: 0.3209 - val_f1_m: 0.4440
Epoch 4/120
 - 1859s - loss: 0.6559 - precision_m: 0.5993 - recall_m: 0.7190 - f1_m: 0.6455 - val_loss: 0.6486 - val_precision_m: 0.5788 - val_recall_m: 1.0000 - val_f1_m: 0.7284
Epoch 5/120
 - 1859s - loss: 0.6547 - precision_m: 0.5987 - recall_m: 0.7328 - f1_m: 0.6515 - val_loss: 0.6263 - val_precision_m: 0.8457 - val_recall_m: 0.5185 - val_f1_m: 0.6336
Epoch 6/120
 - 1865s - loss: 0.6550 - precision_m: 0.6017 - recall_m: 0.7223 - f1_m: 0.6483 - val_loss: 0

# Trial 1

- 2 Convolutional Layers and 1 Fully-Connected Layer
- No Regularization
- Best Validation Loss: 0.53 (Epoch 7/35)
- ~20 mins per epoch

```
classifier = Sequential()
classifier.add(Conv2D(32, (5, 5), input_shape=(200, 200, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

classifier.add(Dense(activation="relu", units=128))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

# Trial 2

- 6 convolutional layers with a max pooling layer every 2 convolutions
- Little improvement in validation/training loss and accuracy over the first hour and a half of training
- Best validation loss: 0.69 (last epoch for stopping)
- ~20 mins/epoch

```
classifier = Sequential()

# 200x200 input
# Input is kinda large, so we select larger filters for the first layer to decrease 
# size of feature maps (and hopefully speed up training).

# Input: 100 x 100 x 1
classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), activation='relu'))
# Input: 96 x 96 x 32
classifier.add(Conv2D(64, (3, 3), activation='relu'))
# Input: 94 x 94 x 64
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 47 x 47 x 64
classifier.add(Conv2D(128, (3, 3), activation='relu'))
# Input: 45 x 45 x 128
classifier.add(Conv2D(128, (3, 3), activation='relu'))
# Input: 43 x 43 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 21 x 21 x 128
classifier.add(Conv2D(128, (3, 3), activation='relu'))
# Input: 19 x 19 x 128
classifier.add(Conv2D(128, (3, 3), activation='relu'))
# Input: 17 x 17 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 8 x 8 x 128
classifier.add(Flatten())

# Input: 8192 x 512
classifier.add(Dense(activation="relu", units=512))
classifier.add(Dense(activation="relu", units=512))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

# Trial 3

- 2 convolutional layers with max pooling after every layer
- 2 512-unit fully-connected layers
- Best validation loss: 0.45 (Epoch 21/30)
- ~16 mins per epoch

```
classifier = Sequential()
classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

classifier.add(Dense(activation="relu", units=512))
classifier.add(Dense(activation="relu", units=512))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

# Trial 4

- Same as Trial 3 but with a batch normalization layer and 256-unit fully-connected layers
- Best validation loss: 0.482 (Epoch 8/30)
- ~16 mins per epoch

```
classifier = Sequential()

classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), activation='relu', use_bias=False))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

classifier.add(Dense(activation="relu", units=256))
classifier.add(Dense(activation="relu", units=256))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

# Trial 5

- 6 convolutional layers and 2 fully-connected layers
- Batch normalization after every convolutional layer
- ReLU activation should be after pooling layer for performance (this was a mistake)
- Dropout of 0.2 after every convolutional layer (this invalidates a random 20% of input units every training iteration).  This should help with overfitting
- ~33 minutes per epoch
- Best validation loss: 0.4183 (Epoch 35/65) 

```
classifier = Sequential()

# L2 regularization uses the sum of the squares of the weights
l2_regularization_constant = 0

# 200x200 input
# Input is kinda large, so we select larger filters for the first layer to decrease 
# size of feature maps (and hopefully speed up training).

# Input: 100 x 100 x 1
classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), use_bias=False))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 96 x 96 x 32
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 94 x 94 x 64
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 47 x 47 x 64
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 45 x 45 x 128
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 43 x 43 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 21 x 21 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 19 x 19 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 17 x 17 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 8 x 8 x 128
classifier.add(Flatten())

# Input: 8192 x 1024
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```